<a href="https://colab.research.google.com/github/gollum-98/NLP_Script_Generator/blob/master/Star_Wars_Script_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
with open("/content/SW_.txt") as f:
  data = f.readlines()
#print(data)

In [ ]:
tokenizer = Tokenizer(oov_token="<UNK>",filters = '!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t')
tokenizer.fit_on_texts(data)
word_index = tokenizer.word_index
print(len(word_index))

In [ ]:
print(word_index)

In [ ]:
input_sequences = []
for line in data:
  token_list = tokenizer.texts_to_sequences([line])[0]
  for i in range (1,len(token_list)):
    n_gram = token_list[:i+1]
    input_sequences.append(n_gram)
print(input_sequences)

In [ ]:
##pad sequences
max_length = max([len(x) for x in input_sequences])
x = pad_sequences(input_sequences,maxlen=max_length)
seq = np.array(x)

In [ ]:
xs, labels = seq[:,:-1],seq[:,-1]

In [ ]:
print(xs.shape)

In [ ]:
print(labels)

In [ ]:
total_words = len(word_index) + 1
from tensorflow.keras.utils import to_categorical
ys = to_categorical(labels,total_words)

In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/glove.6B.100d.txt \
    -O /tmp/glove.6B.100d.txt
embeddings_index = {};
with open('/tmp/glove.6B.100d.txt') as f:
    for line in f:
        values = line.split();
        word = values[0];
        coefs = np.asarray(values[1:], dtype='float32');
        embeddings_index[word] = coefs;

embeddings_matrix = np.zeros((total_words, 100));
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word);
    if embedding_vector is not None:
        embeddings_matrix[i] = embedding_vector;

In [ ]:
model = tf.keras.models.Sequential([
                                    tf.keras.layers.Embedding(total_words,100,input_length=max_length-1,weights=[embeddings_matrix], trainable=False),
                                    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(512,return_sequences=True)),
                                    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(512)),
                                    tf.keras.layers.Dense(total_words*2,activation="relu"),
                                    tf.keras.layers.Dense(total_words,activation="softmax")
])
model.summary()
model.compile(loss="categorical_crossentropy",optimizer=tf.keras.optimizers.Adam(),metrics=["accuracy"])

In [ ]:
history = model.fit(xs,ys,epochs=70,batch_size=256)

In [ ]:
seed_text = 'character dialogue\nLuke This isn\'t the droid'
next_words = 100
  
for _ in range(next_words):
	token_list = tokenizer.texts_to_sequences([seed_text])[0]
	token_list = pad_sequences([token_list], maxlen=max_length-1, padding='pre')
	predicted = model.predict_classes(token_list, verbose=0)
	output_word = ""
	for word, index in tokenizer.word_index.items():
		if index == predicted:
			output_word = word
			break
	seed_text += " " + output_word
print(seed_text)